In [ ]:
import pickle
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import cm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy.spatial import distance_matrix
def smooth(x, y, h=1):
    K = np.exp(-distance_matrix(x.values.reshape(-1,1), x.values.reshape(-1,1))**2/(2*h))
    return (K@y) / (K@np.ones_like(y))

In [ ]:
X_train, X_test, y_train, y_test = pickle.load(open('mnist.pkl', 'rb'))

In [ ]:
results = pickle.load(open('results_different_lambdas.pkl', 'rb')) + pickle.load(open('results_end_to_end.pkl', 'rb'))
result_df = pd.DataFrame.from_dict(results)
result_df = result_df.query('step != 284515')

result_df['L'] = 1
result_df['h'] = result_df['N']
result_df['N'] = result_df['h']*(result_df['d'] + 1)

force = lambda y,f: 1 - y*f
loss = lambda y,f: np.mean(np.maximum(0, force(y,f))**2, -1)
N_del = lambda y,f: np.sum(force(y,f) >= 0, -1)

result_df['test_loss'] = result_df.y_test_hat.apply(lambda f: loss(y_test, f))
result_df['train_loss'] = result_df.y_train_hat.apply(lambda f: loss(y_train, f))
result_df['N_del'] = result_df.y_train_hat.apply(lambda f: N_del(y_train, f))

result_df['P/N'] = result_df['P']/result_df['N']
result_df['N_del/N'] = result_df['N_del']/result_df['N']

result_df['P/h'] = result_df['P']/result_df['h']
result_df['N_del/h'] = result_df['N_del']/result_df['h']

result_end_to_end_df = result_df[result_df['lambda'] == 0]
result_df = result_df[result_df['lambda'] > 0]

In [ ]:
star_cutoff = 5e-2

N_star = result_df.groupby('step').apply(lambda df: df.query('`lambda` == 1e-15').query('(train_loss > @star_cutoff)')['N'].max())
result_df['N_star'] = result_df['step'].map(N_star)

h_star = result_df.groupby('step').apply(lambda df: df.query('`lambda` == 1e-15').query('(train_loss > @star_cutoff)')['h'].max())
result_df['h_star'] = result_df['step'].map(h_star)

In [ ]:
N_star = result_end_to_end_df.groupby('step').apply(lambda df: df.query('(train_loss > @star_cutoff)')['N'].max())
result_end_to_end_df['N_star'] = result_end_to_end_df['step'].map(N_star)

h_star = result_end_to_end_df.groupby('step').apply(lambda df: df.query('(train_loss > @star_cutoff)')['h'].max())
result_end_to_end_df['h_star'] = result_end_to_end_df['step'].map(h_star)

In [ ]:
from matplotlib import colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list(
    'Mei2019', 
    np.array([
        (243, 232, 29),
        (245, 173, 47),
        (140, 193, 53),
        (50,  191, 133),
        (23,  167, 198),
        (36,  123, 235),
        (53,  69,  252),
        (52,  27,  203)
    ])/255., 
    N=256
).reversed()

# cmap = cc.m_bmy

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
fig = plt.figure(figsize=(6,.5))
img = plt.imshow(gradient, aspect='auto', cmap=cmap)
title = plt.title('Colormap stolen from Mei2019')

norm=mcolors.LogNorm()

In [ ]:
SMALL_SIZE = 11
MEDIUM_SIZE = 12
BIGGER_SIZE = 14
BIGGEST_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGEST_SIZE)  # fontsize of the figure title

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'N_del/h'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step > 1e6')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"$N_\Delta/h$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'N_del/h'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step == 0')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"$N_\Delta/h$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'train_loss'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step > 1e6')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"Train $\mathcal{L}$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'train_loss'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step == 0')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"Train $\mathcal{L}$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'test_loss'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step > 1e6')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"Test $\mathcal{L}$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')

In [ ]:
fig = plt.figure(figsize=(9,6))
cmap = cm.plasma_r

by = '`lambda`'
x_expr = 'h/h_star'
y_expr = 'test_loss'

sm = cm.ScalarMappable(norm=norm, cmap=cmap)

by_vals = np.array(sorted(result_df.eval(by).unique(), reverse=False)) 
#random zorder helps with visual clarity 
extremes = by_vals[[0, -1]]
by_vals = by_vals[1:-1]
by_vals = np.random.choice(by_vals, size=len(by_vals), replace=False)
by_vals = np.append(by_vals, extremes)
norm.autoscale(by_vals)

for val in by_vals: #[::-1]:
    color = cmap(norm(val))
    
    data = result_df.query('step == 0')
    data = data.query(f'{by} == @val')
    data = data.iloc[np.argsort(data.eval(x_expr))]
    
    x = data.eval(x_expr)
    y = data.eval(y_expr)
    plt.scatter(x, y, c=data.eval(by), cmap=cmap, norm=norm, alpha=.7)
    
#     y_sm = smooth(np.log(x), y, .001)
#     plt.plot(x, y_sm, color=color, ls=':')

plt.colorbar(sm, label=r'$\lambda$')
plt.xlabel(x_expr)
plt.ylabel(y_expr)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0, 2)
plt.axhline(1, c='k', ls='--', alpha=.7)
plt.axvline(1, c='k', ls='--', alpha=.7)


# data = result_end_to_end_df
# x = data.eval(x_expr)
# y = data.eval(y_expr)
# plt.scatter(x, y, c='k', marker='x')

plt.xlabel(r"$h/h^*$")
plt.ylabel(r"Test $\mathcal{L}$")
# fig.savefig('plots/h_h_star_vs_N_del_h_L=1.pdf')